In [22]:
import numpy as np
import scipy

In [23]:
dataset1 = np.random.randint(1, 100, size=100)
dataset2 = np.random.randint(1, 100, size=100)

In [46]:
def get_score(data1: np.ndarray, data2: np.ndarray) -> tuple[int, int]:
    diff = data1 - data2
    diff = diff[diff != 0]

    ranks = np.abs(diff).argsort()

    # 各符号ごとに順位の和を計算
    positive_rank_sum = ranks[diff > 0].sum()
    negative_rank_sum = ranks[diff < 0].sum()

    return np.min([positive_rank_sum, negative_rank_sum]), len(diff)

def get_p_value(score: int, N: int) -> float:
    # wilcoxon分布はサンプル量が多いとき、中心極限定理により正規分布に近似できる
    # wilcoxonの期待値と分散を計算した上で、正規分布の累積分布関数を使うことで、p値を計算する

    # 期待値: E[T] = n(n+1)/4
    # 分散: Var[T] = n(n+1)(2n+1)/24
    E = N * (N + 1) / 4
    V = N * (N + 1) * (2 * N + 1) / 24

    low = False
    if E > score:
        low = True
    
    # cdfは、与えられた引数xをx軸にとった時の、x以下の面積を返す
    # xがEより大きい場合は、上裾の面積になる
    cdf = scipy.stats.norm.cdf(score, loc=E, scale=np.sqrt(V))

    # 両側検定なので*2する
    if low:
        return cdf * 2
    else:
        return (1 - cdf) * 2

In [44]:
score, N = get_score(dataset1, dataset2)
p_value = get_p_value(score, N)

print(f"p_value: {p_value}")

p_value: 0.4764529395105843


In [45]:
if p_value > 0.05:
    print('dataset1 と dataset2 の間に有意差はありません')
else:
    print('dataset1 と dataset2 の間に有意差があります')

dataset1 と dataset2 の間に有意差はありません
